In [163]:
from functools import cmp_to_key
import os
import pandas as pd

In [164]:
def compare_file(file1, file2):
    num1 = int(file1.split('.')[0])  
    num2 = int(file2.split('.')[0])  
    if num1 < num2:
        return -1
    elif num1 > num2:
        return 1
    else:
        return 0
def pb_read_appendix1(path="./Problem B/Appendix 1"):
   data=[]
   for (_, _, filenames) in os.walk(path):
      temp_save=filenames.copy()
      for idx, ff in enumerate(filenames):
          if ff.startswith("~"):
              temp_save.remove(ff)
      filenames=sorted(temp_save, key=cmp_to_key(compare_file))
      for filename in filenames:
         excel_instance=pd.read_excel(os.path.join(path, filename), sheet_name='Sheet1')
         data.append({})
         data[-1]["date"]=[i for i in excel_instance["Var1"].to_list()]
         data[-1]["time"]=[i for i in excel_instance["Var2"].to_list()]
         data[-1]["x"]=[i for i in excel_instance["Var3"].to_list()]
         data[-1]["y"]=[i for i in excel_instance["Var4"].to_list()]
         data[-1]["z"]=[i for i in excel_instance["Var5"].to_list()]
         print(f"{filename} len:{len(data[-1]["date"])}")
   lower_range=555 
   upper_range=1033
   for sensor in data:
       sensor["date"]=sensor["date"][lower_range:upper_range+1]
       sensor["time"]=sensor["time"][lower_range:upper_range+1]
       sensor["x"]=sensor["x"][lower_range:upper_range+1]
       sensor["y"]=sensor["y"][lower_range:upper_range+1]
       sensor["z"]=sensor["z"][lower_range:upper_range+1]
   return data

In [165]:
appendix1=pb_read_appendix1() #读取所有传感器的数据到一个字典里 只取24:00到4:00的数据！

1.xlsx len:4310
2.xlsx len:4310
3.xlsx len:4310
4.xlsx len:4310
5.xlsx len:4310
6.xlsx len:4310
7.xlsx len:4310
8.xlsx len:4310
9.xlsx len:4310
10.xlsx len:4310
11.xlsx len:4310
12.xlsx len:4310
13.xlsx len:4310
14.xlsx len:4310
15.xlsx len:4310
16.xlsx len:4310
17.xlsx len:4310
18.xlsx len:4310
19.xlsx len:4310
20.xlsx len:4310
21.xlsx len:4310
22.xlsx len:4310
23.xlsx len:4310
24.xlsx len:4310
25.xlsx len:4310
26.xlsx len:4310
27.xlsx len:4310
28.xlsx len:4310
29.xlsx len:4310
30.xlsx len:4310
31.xlsx len:4310
32.xlsx len:4310
33.xlsx len:4310
34.xlsx len:4310
35.xlsx len:4310
36.xlsx len:4310
37.xlsx len:4310
38.xlsx len:4310
39.xlsx len:4310
40.xlsx len:4310
41.xlsx len:4310
42.xlsx len:4310
43.xlsx len:4310
44.xlsx len:4310
45.xlsx len:4310
46.xlsx len:4310
47.xlsx len:4310
48.xlsx len:4310
49.xlsx len:4310
50.xlsx len:4310
51.xlsx len:4310
52.xlsx len:4310
53.xlsx len:4310
54.xlsx len:4310
55.xlsx len:4310
56.xlsx len:4310
57.xlsx len:4310
58.xlsx len:4310
59.xlsx len:4310
60.xls

# 读取格式
读取出来是一个array,每个array是每个传感器的信息
## keys 
1. date$\Rightarrow$一个包含不同点日期的array
2. time$\Rightarrow$一个包含对应时间戳的点
3. x$\Rightarrow$该时间戳下的x值
4. y$\Rightarrow$该时间戳下的y值
5. z$\Rightarrow$该时间戳下的z值

In [171]:
import torch 
import torch.nn as nn
import numpy as np
from torch.utils.data import DataLoader, Dataset
device="cuda"

In [172]:
xs=[]
ys=[]
zs=[]
for sensor in appendix1:
    xs.extend([x-sensor["x"][0] for x in sensor["x"]])
    ys.extend([y-sensor["y"][0] for y in sensor["y"]])
    zs.extend([z-sensor["z"][0] for z in sensor["z"]])
xs=np.array(xs)
ys=np.array(ys)
zs=np.array(zs)
xs=torch.from_numpy(xs).to(device)
ys=torch.from_numpy(ys).to(device)
zs=torch.from_numpy(zs).to(device)
output_size=xs.size(0)
print(output_size)

169087


In [168]:
class Loss(nn.Module):
    def compute_mse(self,x):
        n = x.size(0)    
    # 计算均方差
        mse = torch.zeros(n).to(device) # 沿着第二个维度（列）求均值
        for idx in range(n):
            temp_mat=x[idx]*torch.ones(n).to(device)
            this_mse=torch.mean((temp_mat-x)**2)
            mse[idx]=this_mse
        return mse

    def __init__(self, all_xyz:dict):
        super(Loss, self).__init__()
        self.all_xyz=all_xyz
        self.mse_x=torch.from_numpy(np.load("mse_x.npy")).to(device)
        self.mse_y=torch.from_numpy(np.load("mse_y.npy")).to(device)
        self.mse_z=torch.from_numpy(np.load("mse_z.npy")).to(device)
    def forward(self, pred_output, fake_label):
        loss_x=torch.mean(torch.multiply(pred_output, self.mse_x))
        loss_y=torch.mean(torch.multiply(pred_output, self.mse_y))
        loss_z=torch.mean(torch.multiply(pred_output, self.mse_z))
        return ((loss_x+loss_y+loss_z)/3)
    
        

In [169]:
criterion=Loss({"x":xs,"y":ys,"z":zs})

In [195]:
combined_mse=(criterion.mse_x+criterion.mse_y+criterion.mse_z)/3
print(torch.argmin(combined_mse), torch.min(combined_mse))
min_idx=torch.argmin(combined_mse).item()
sensor_idx=int(min_idx/len(appendix1[0]["date"]))
print(sensor_idx)
ts_idx=min_idx-sensor_idx*len(appendix1[0]["date"])
print(ts_idx)
print(f"Sensor:{sensor_idx}, time:{appendix1[sensor_idx]["time"][ts_idx]},x:{xs[min_idx]},y:{ys[min_idx]},z:{zs[min_idx]}")
print(appendix1[sensor_idx]["x"])

tensor(104136, device='cuda:0') tensor(5.0577, device='cuda:0')
217
193
Sensor:217, time:01:37:06,x:-0.17100000000209548,y:-0.4479999999999986,z:0.06199999999807915
[30972.363, 30972.437, 30972.49, 30972.503, 30972.563, 30972.415, 30972.345, 30972.298, 30972.298, 30972.243, 30972.202, 30972.303, 30972.31, 30972.262, 30972.239, 30972.206, 30972.253, 30972.37, 30972.38, 30972.353, 30972.327, 30972.325, 30972.339, 30972.376, 30972.356, 30972.525, 30972.475, 30972.528, 30972.456, 30972.567, 30972.599, 30972.505, 30972.386, 30972.329, 30972.267, 30972.299, 30972.341, 30972.263, 30972.247, 30972.267, 30972.34, 30972.296, 30972.372, 30972.341, 30972.399, 30972.495, 30972.455, 30972.409, 30972.372, 30972.396, 30972.409, 30972.55, 30972.548, 30972.497, 30972.101, 30972.106, 30972.162, 30972.143, 30972.189, 30972.171, 30972.125, 30972.13, 30972.112, 30972.156, 30972.175, 30972.327, 30972.323, 30972.214, 30972.236, 30972.237, 30972.304, 30972.309, 30972.404, 30972.407, 30972.407, 30972.432, 30972

In [189]:
min_idx-217*479

193